In [1]:
!pip install git+https://github.com/huggingface/diffusers.git
!pip install transformers scipy ftfy
!pip install "ipywidgets>=7,<8"
!pip install accelerate


  Cloning https://github.com/huggingface/diffusers.git to /tmp/pip-req-build-v7ek7li2
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/diffusers.git /tmp/pip-req-build-v7ek7li2
  Resolved https://github.com/huggingface/diffusers.git to commit 31444f5790a8f91fc2caf3995fadc561e23fb279
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
import torch
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler
from PIL import Image

device = "cuda"
model_id = "./sd-2-1/stable-diffusion-2-1"

pipe = StableDiffusionPipeline.from_pretrained(model_id, revision="fp16", torch_dtype=torch.float16, use_auth_token=True)
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)

pipe = pipe.to(device)

prompt = "anime girl, beautiful, big breast, blue eyes, long hair, masterpiece"
num_inference_steps = 100
height = 768
width = 768
num_images_per_prompt = 1
guidance_scale = 7.5


generator = torch.Generator(device=device)
latents = None
seeds = []
for _ in range(num_images_per_prompt):
    # Get a new random seed, store it and use it as the generator state
    seed = generator.seed()
    seeds.append(seed)
    generator = generator.manual_seed(seed)
    
    image_latents = torch.randn(
        (1, pipe.unet.in_channels, height // 8, width // 8),
        generator = generator,
        dtype=torch.float16,
        device = device
    )
    latents = image_latents if latents is None else torch.cat((latents, image_latents))
    
# import pdb;pdb.set_trace()
PipelineOut = pipe(prompt, num_inference_steps=num_inference_steps, guidance_scale=guidance_scale, height=height, width=width, num_images_per_prompt=num_images_per_prompt, latents=latents)
images = PipelineOut.images 

for i in range(num_images_per_prompt):
    images[i].save("./images/image_{}_{}.png".format(str(i), str(seeds[i])))

  0%|          | 0/100 [00:00<?, ?it/s]

In [3]:
seed = seeds[0]   
generator.manual_seed(seed)

latents = torch.randn(
    (1, pipe.unet.in_channels, height // 8, width // 8),
    generator = generator,
    dtype=torch.float16,
    device = device
)
PipelineOut = pipe(prompt, num_inference_steps=num_inference_steps, guidance_scale=guidance_scale, height=height, width=width, latents=latents)
images = PipelineOut.images
images[0].save("./images/final.png")

  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
import time;time.sleep(7200)